In [ ]:
##### Libraries
#---------------------------------------------------------------------
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from datetime import datetime
from tqdm import tqdm
from sklearn.neighbors import BallTree
from sklearn.linear_model import SGDClassifier



In [ ]:
#!pip install '../input/translate/translate-3.6.1.tar'
#!pip install translate
#from translate import Translator

In [ ]:
##### Data Load
#---------------------------------------------------------------------
start = datetime.now()

path='../input/foursquare-location-matching/'
df_train=pd.read_csv(path + 'train.csv')
df_test=pd.read_csv(path + 'test.csv')
df_pairs=pd.read_csv(path + 'pairs.csv')
df_submission=pd.read_csv(path + 'sample_submission.csv')
df_ISO=pd.read_csv('../input/isonum/ISONum.csv',encoding="ISO-8859-2")

In [ ]:
#### Missing Values
#---------------------------------------------------------------------
df_train.loc[df_train.country.isnull()]
df_pairs.country_1.loc[df_pairs.country_1=='Windhoek']
df_pairs.country_2.loc[df_pairs.country_2=='Windhoek']
df_train['country'].fillna('NA',inplace=True)

df_pairs.loc[df_pairs.country_1.isnull()]
df_pairs.loc[df_pairs.country_2.isnull()]
df_pairs['country_1'].fillna('NA',inplace=True)
df_pairs['country_2'].fillna('NA',inplace=True)

df_test.categories=df_test.categories.fillna('__NAN__')
df_test.name = df_test.name.fillna('__NAN__')

df_train.categories=df_train.categories.fillna('__NAN__')
df_train.name = df_train.name.fillna('__NAN__')

In [ ]:
df=df_test.append(df_train.drop('point_of_interest',axis=1),ignore_index=True)

In [ ]:
#### Model & Evaluation
#---------------------------------------------------------------------
tree = BallTree(np.deg2rad(df[['latitude', 'longitude']].values), metric='haversine')

# list for storing the points of interest
pois_out = []

max_dist_cat = 0.00008
max_dist_name = 0.005
distance=0.00018 #equivalente to about 1.200 KM

#translator=Translator(to_lang='en')
#df.name=df['name'].apply(lambda s: translator.translate(s))

for i, row in tqdm(df_test.iterrows()):
    #dist, ind = tree.query_radius(np.deg2rad(np.c_[row['latitude'], row['longitude']]), r=distance,return_distance=True)
    ind = tree.query_radius(np.deg2rad(np.c_[row['latitude'], row['longitude']]), r=distance)
    poi = []
    for j in ind[0]:
    #for d, j in zip(dist[0], ind[0]):
        #categories=translator.translate(row['categories'])
        #categories=row['categories']
        #df_categories=translator.translate(df.categories.iloc[j])
        #df_categories=df.categories.iloc[j]
        if row['categories'] != '__NAN__' and (row['categories'] in df.categories.iloc[j] or df.categories.iloc[j] in row['categories']):
        #if d <= max_dist_cat and row['categories'] != '__NAN__' and (row['categories'] in df.categories.iloc[j] or df.categories.iloc[j] in row['categories']):
        #if categories != '__NAN__' and (categories in df_categories or df_categories in categories):
            #aux = pd.DataFrame([[row['id'],row['latitude'],row['longitude'] ,df.id.iloc[j],df.latitude.iloc[j],df.longitude.iloc[j], 1]], columns=['id_1', 'latitude_1','longitude_1','id_2','latitude_2','longitude_2', 'match'])
            #X=aux.drop(['match','id_1','id_2'],axis=1)
            #y_hat=classifier.predict(X)
            #if y_hat==1:
            poi.append(df.id.iloc[j])
            #df_hat=df_hat.append(aux)
        #elif d <= max_dist_name and row['name'] != '__NAN__' and (row['name'].lower() == df.name.iloc[j].lower()):
        elif row['name'] != '__NAN__' and (row['name'].lower() == df.name.iloc[j].lower()):
            #aux = pd.DataFrame([[row['id'], row['latitude'], row['longitude'], df.id.iloc[j], df.latitude.iloc[j],df.longitude.iloc[j], 1]], 
                               #columns=['id_1', 'latitude_1', 'longitude_1', 'id_2', 'latitude_2', 'longitude_2',
                                        #'match'])
            #X = aux.drop('match', axis=1)
            #y_hat = classifier.predict(X)
            #if y_hat==1:
            poi.append(df.id.iloc[j])
            #df_hat = df_hat.append(aux)

    if len(poi) == 0:
        pois_out.append(row['id'])
    else:
        pois_out.append(' '.join(poi))


In [ ]:
##### Convert prediction in submission format
#---------------------------------------------------------------------
submission=pd.DataFrame(columns=['id','matches'])
submission['id']=df_test['id']
submission['matches']=pois_out

submission.to_csv("submission.csv",index=False)
print(submission)